# Retrieval-Based Chatbot with BOW Intent Classification & Word Embeddings for Entity Recognition

### Practice: CityBot Speaks Italian

Importing NLP libraries and tools; setting up corpus of stop words, word embeddings, and lemmatizer for Italian.

In [1]:
import re
from collections import Counter
import spacy
word2vec = spacy.load("it_core_news_md")
import pprint
import treetaggerwrapper
from nltk.corpus import stopwords
italian_stop_words = set(stopwords.words("italian"))
tagger = treetaggerwrapper.TreeTagger(TAGLANG="it")

C:\workspace\miniconda3\lib\site-packages\treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
C:\workspace\miniconda3\lib\site-packages\treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
C:\workspace\miniconda3\lib\site-packages\treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
C:\workspace\miniconda3\lib\site-packages\treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


Defining variables with the answers that will be retrieved and the word that corresponds to the one entity ("city") that will be captured by the bot. Defining a set of exit commands as well.

In [2]:
slot = "città"

answer_1 = """
Agli abitanti di {} piace andare al mare durante le vacanze.
Prendono il sole, si rilassano, e si divertono,
benché la spiaggia sia piena di spazzatura."""

answer_2 = """
Il costo della vita è molto alto a {}.
Gli affitti sono assai cari; e chi fischietta per strada deve pagare multe salate."""

answer_3 = """
{} è la mia citta preferita!
Ci sono tante persone simpatiche e puoi andare in giro nudo."""

answer_4 = """
Ti consiglio di visitare {}, se ti piace la cultura.
C'è uno straordinario museo del formaggio."""

answer_5 = """
Se cerchi tante aree verdi, {} è il posto giusto per te.
Ci sono parecchi parchi, orti botanici, giardini, e foreste piene di criceti."""

answer_6 = """
Non avere paura, a {} c'è pochissimo crimine.
Nessun furto, niente vandalismo, solo costanti sparatorie contro i piccioni."""

answer_7 = """
Quando sono a {}, l'attività che mi piace di più è andare al cinema!
Ci sono anche un paio di teatri, ma lì non puoi mangiare il popcorn caduto per terra."""

answer_8 = """
Il palazzo più famoso di {} è la Torre del Puzzo.
Contiene il celebre monumento al Merluzzo Peloso, una vera meraviglia dell'arte."""

answer_9 = """
Tutto ciò che posso dirti a proposito degli abitanti di {},
è che adorano i ristoranti che servono il risotto con le verze."""

answers = [answer_1, answer_2, answer_3, answer_4, answer_5, answer_6, answer_7, answer_8, answer_9]

exit_commands = ["ciao", "arrivederci", "niente", "basta", "vado", "esci"] 

Creating preprocessing function to make the text lowercase, remove punctuation and stop words, and lemmatizing it.

In [3]:
def preprocess(text):
    lowercase_text = text.lower()
    cleaned_text = re.sub(r"[^\w\s]", " ", lowercase_text)
    tagged_text = treetaggerwrapper.make_tags(tagger.tag_text(cleaned_text))
    lemmatized_text = [lemma for word, tag, lemma in tagged_text if lemma not in italian_stop_words]
    return lemmatized_text

Two of the following three functions determine the similarity between two strings using either a BOW model (`compare_overlap`) or word embeddings vectors (`compute_similarity`). The third function (`extract_nouns`) tags a string in Italian for POS and extracts its nouns.

In [4]:
def compare_overlap(user_input_bow, possible_answer_bow):
    similar_words_score = 0
    for word in user_input_bow:
        if word in possible_answer_bow:
            similar_words_score += 1
    return similar_words_score

def extract_nouns(text):
    tagged_text = treetaggerwrapper.make_tags(tagger.tag_text(text))
    nouns = [lemma for word, tag, lemma in tagged_text if tag == "NPR"]
    return nouns

def compute_similarity(possible_entities, slot):
    similarity_list = [(possible_entity, slot, possible_entity.similarity(slot)) for possible_entity in possible_entities]
    return similarity_list

Finally, the Chatbot is defined as a class object which takes the user's input and either continues or interrupts a conversation, selecting and delivering the most appropriate answers.

In [5]:
class CityBotto:
    
    # Defining function to stop conversation if the user enters an exit command.
    def stop_convo(self, user_input):
        for command in exit_commands:
            if command in user_input.lower():
                print("Va bene, alla prossima! Tanti saluti e un bel bacione.")
                return True
            
    # Defining function to start conversation and keep it going until the user enters an exit command.
    def start_convo(self):
        greeting = """
Ciao, sono Citybotto!
Dispongo di tante informazioni su tutte le città del mondo.
Fammi una domanda riguardo alla tua città.\n> """
        user_input = input(greeting)
        while not self.stop_convo(user_input):
            user_input = self.retrieve_answer(user_input)

    # Defining function to classify intent of the user input.
    def classify_intent(self, answers, user_input):
        user_input_bow = Counter(preprocess(user_input))
        possible_answers_bow = [Counter(preprocess(answer)) for answer in answers]
        list_of_similarity_scores = [compare_overlap(answer_bow, user_input_bow) for answer_bow in possible_answers_bow]
        best_answer_index = list_of_similarity_scores.index(max(list_of_similarity_scores))
        return answers[best_answer_index]
    
    # Defining function to recognize entities in the user input.
    def recognize_entity(self, user_input):
        user_input_nouns = extract_nouns(user_input)
        embeddings_possible_entities = word2vec(" ".join(user_input_nouns))
        embeddings_slot = word2vec(slot)
        similarity_scores = compute_similarity(embeddings_possible_entities, embeddings_slot)
        # The following arranges the similarity scores in ascending order.
        similarity_scores.sort(key=lambda x: x[2])
        # Checks if the similarity scores list is empty; if it is not,
        # it returns the entity that corresponds to the highest score
        # (the first value in the last tuple contained in the list);
        # if it is, it simply returns the base value of the slot.
        try:
            return similarity_scores[-1][0]
        except IndexError:
            return slot
    
    # Defining function to retrieve answer to the user based on the user's input and to ask for another question.
    def retrieve_answer(self, user_input):
        best_answer = self.classify_intent(answers, user_input)
        entity = self.recognize_entity(user_input)
        print(best_answer.format(entity))
        user_input = input("Vuoi farmi un'altra domanda riguardo a una città?\n> ")
        return user_input

In [6]:
city_bot = CityBotto()
city_bot.start_convo()


Ciao, sono Citybotto!
Dispongo di tante informazioni su tutte le città del mondo.
Fammi una domanda riguardo alla tua città.
> Sono un abitante di Parigi. Ti piace questa città?

Agli abitanti di Parigi piace andare al mare durante le vacanze.
Prendono il sole, si rilassano, e si divertono,
benché la spiaggia sia piena di spazzatura.
Vuoi farmi un'altra domanda riguardo a una città?
> Roma è la tua città preferita?

Roma è la mia citta preferita!
Ci sono tante persone simpatiche e puoi andare in giro nudo.
Vuoi farmi un'altra domanda riguardo a una città?
> Ci sono buoni ristoranti a Berlino?

Tutto ciò che posso dirti a proposito degli abitanti di Berlino,
è che adorano i ristoranti che servono il risotto con le verze.
Vuoi farmi un'altra domanda riguardo a una città?
> Posso andare al cinema a Vladivostok?

Quando sono a città, l'attività che mi piace di più è andare al cinema!
Ci sono anche un paio di teatri, ma lì non puoi mangiare il popcorn caduto per terra.
Vuoi farmi un'altra 